In [80]:
import keras
import librosa
import librosa.display
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import random
import warnings
warnings.filterwarnings('ignore')
import string
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [2]:
np.random.seed(1337)
random.seed(1337)

In [8]:
meta_data = pd.read_csv('./ESC-50-master/meta/esc50.csv')
meta_data.head()

,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A


In [4]:
filenames = meta_data['filename']
labels = meta_data['target']

In [5]:
x_train_val, x_test, y_train_val, y_test = train_test_split(filenames, labels, random_state=1337, stratify=labels, test_size=.15)
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, random_state=1337, stratify=y_train_val, test_size=0.117647058823529)

In [6]:
human_data = pd.read_csv('./human.csv')

In [7]:
human_data.head()

,Prediction,Truth,Recording
0,Dog,Dog,1-100032-A.ogg
1,Chirping birds,Chirping birds,1-100038-A.ogg
2,Hand saw,Vacuum cleaner,1-100210-A.ogg
3,Vacuum cleaner,Vacuum cleaner,1-100210-B.ogg
4,Thunderstorm,Thunderstorm,1-101296-A.ogg


In [95]:
label_map = {}

for row in human_data.iterrows():
    if row[1].Prediction == row[1].Truth:
        matched_row = meta_data[meta_data['filename'].str.contains(row[1].Recording[:10])]
        
        if matched_row.empty:
            continue
        
        label_map[row[1].Prediction] = int(matched_row.iloc[0]['target'])

In [96]:
len(label_map)

50

In [97]:
human_test = []
ground_truth_human_test = []
test_file_names = np.array(x_test)

for row in human_data.iterrows():
    matched_row = meta_data[meta_data['filename'].str.contains(row[1].Recording[:10])]
    
    if matched_row.empty or matched_row.iloc[0]['filename'] not in test_file_names:
        continue
    
    human_test.append(label_map[row[1].Prediction])
    ground_truth_human_test.append(label_map[row[1].Truth])

In [100]:
def print_metrics(predictions, groun_truth):
    average_type = 'macro'

    accuracy = accuracy_score(predictions, groun_truth)
    precision = precision_score(predictions, groun_truth, average=average_type)
    recall = recall_score(predictions, groun_truth, average=average_type)
    f1 = f1_score(predictions, groun_truth, average=average_type)
    
    print('acc: ', accuracy)
    print('pre: ', precision)
    print('rec: ', recall)
    print('f1 : ', f1)

In [101]:
print_metrics(human_test, ground_truth_human_test)

acc:  0.8415672913117547
pre:  0.8361194083694085
rec:  0.8522059035735506
f1 :  0.8335958927180106
